In [1]:
# Import the library and APIs
import pandas as pd
import seaborn as sns
import matplotlib as mp1
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import csv
import array as arr
import tensorflow as tf
import random
import timeit

from pandas import DataFrame
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from numba import cuda

In [2]:
class Person:
    def __init__(self, id, filename, img, gender, age, angle):
        self.id = id
        self.filename = filename
        self.img = img
        self.gender = gender
        self.age = age
        self.angle = angle
        self.guessGender = 0
        self.guessAge = 0
        self.guessAngle = 0

In [3]:
# Read the .csv file and store in df
# Read the .txt file to get the training id list
%matplotlib inline

df = pd.read_csv('subject_info_OUMVLP.csv')
id_train = np.loadtxt("IDList_train_OUMVLP.txt", dtype='int32')

# Data Mapping

In [5]:
datadir = r"C:/Users/Martin/FYP/GEI_OUMVLP/"
files = os.listdir(datadir)

all_people = []
height = 0 
width = 0

df.ID = df.ID.astype(str)
for index, data in df.iterrows():
    id = (5-len(data.ID))*'0' + data.ID
    gender = 1 if data.gender == 'M' else 0
    age = data.age
    if age == '-':
        age = '-'
    else:
        age = int(age)
    
    temp_id_list = []
    for file in files:
        angle = int(file.split('-')[0])
        sequence = int(file.split('-')[1])
        
        if sequence == 1:
            if id in temp_id_list :
                temp_id_list = []
                continue
            temp_id_list = []
        
        filename = datadir + file + '/' + id + '.png'
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if type(img) == type(None):
            continue
        
        p = Person(id, filename, img, gender, age, angle)
        all_people.append(p)
        temp_id_list.append(id)


In [8]:
category = [0, 15, 30, 45, 60, 75, 90, 180, 195, 210, 225, 240, 255, 270]

all_people_with_angle = {}

for p in all_people:
    if p.angle not in all_people_with_angle:
        all_people_with_angle[p.angle] = []
    all_people_with_angle[p.angle].append(p)


In [7]:
people_train = []
people_test = []

for p in all_people:
    if int(p.id) in id_train:
        people_train.append(p)
    else:
        people_test.append(p)
        
height, width = all_people[0].img.shape

# Model layers without output layer

In [12]:
model = Sequential()
model.add(Conv2D(18, (7, 7), strides = 1, activation = 'relu', input_shape = (height, width, 1)))
model.add(MaxPooling2D(pool_size = (2,2), strides = 2))
model.add(BatchNormalization())

model.add(Conv2D(45, (5, 5), strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides = 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))

model.save("model_MVLP/MVLP_without_output_layer.h5")

# ------------------------------------------------------------------------------

# Training angle

In [13]:
block_angle = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_angle.add(Dense(14, activation = 'softmax'))
block_angle.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_angle.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [14]:
def to_cat(arr):
    y = []
    
    for angle in arr:
        temp = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        index = category.index(angle)
        
        temp[index] = 1
        y.append(temp)
    
    return np.array(y, dtype='int32')

In [15]:
x_train = []
y_train = []

for p in people_train:
    x_train.append(p.img)
    y_train.append(p.angle)
    
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_cat(y_train)

print(y_train.shape)

(68799, 14)


In [16]:
x_test = []
y_test = []

for p in people_test:
    x_test.append(p.img)
    y_test.append(p.angle)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_cat(y_test)

print(y_test.shape)

(69015, 14)


In [17]:
def train(model):
    history = model.fit(x_train, y_train, epochs = 5)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 1)
    return test_acc

In [18]:
acc = 0.9911
epochs = 0

with tf.device("GPU:0"):
    while(True):
        epochs += 50
        new_acc = train(block_angle)
        if new_acc > acc:
            acc = new_acc
            #block_angle.save("model_MVLP/MVLP_angle.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
            print ("\n", b,"\n")

Epoch 1/5
2150/2150 [==============================] - 17s 6ms/step - loss: 0.2854 - accuracy: 0.9439
Epoch 2/5
2150/2150 [==============================] - 13s 6ms/step - loss: 0.0768 - accuracy: 0.9770
Epoch 3/5
2150/2150 [==============================] - 13s 6ms/step - loss: 0.0569 - accuracy: 0.9833
Epoch 4/5
2150/2150 [==============================] - 13s 6ms/step - loss: 0.0428 - accuracy: 0.9874
Epoch 5/5
2157/2157 [==============================] - 5s 2ms/step - loss: 0.0758 - accuracy: 0.9774
Epoch 1/5
2150/2150 [==============================] - 13s 6ms/step - loss: 0.0296 - accuracy: 0.9911
Epoch 2/5
2150/2150 [==============================] - 13s 6ms/step - loss: 0.0257 - accuracy: 0.9928 0s -
Epoch 3/5
2150/2150 [==============================] - 13s 6ms/step - loss: 0.0162 - accuracy: 0.9951
Epoch 4/5
2150/2150 [==============================] - 13s 6ms/step - loss: 0.0158 - accuracy: 0.9953
Epoch 5/5
2157/2157 [==============================] - 5s 2ms/step - loss: 0.0

KeyboardInterrupt: 

In [25]:
with tf.device("GPU:0"):
    block_angle = tf.keras.models.load_model("model_MVLP/MVLP_angle.h5")
    test_loss, test_acc = block_angle.evaluate(x_test, y_test, verbose = 1)

2157/2157 [==============================] - 18s 7ms/step - loss: 0.1091 - accuracy: 0.9911


In [27]:
start = timeit.default_timer()
with tf.device("GPU:0"):
    for p in all_people:
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_angle = block_angle.predict(img)
        p.guessAngle = category[np.argmax(predict_angle)]

        b = "ID: "+ p.id + " Angle: " + str(p.angle) + " Guess: " + str(p.guessAngle)
        print (b, end="\r")
stop = timeit.default_timer()


In [ ]:
print('Prediction Time: ', round((stop - start)/60), 'minutes\n')

In [31]:
train_angle_0_180 = []
train_angle_15_195 = []
train_angle_30_45 = []
train_angle_60_75_90 = []
train_angle_210_225 = []
train_angle_240_255_270 = []

test_angle_0_180 = []
test_angle_15_195 = []
test_angle_30_45 = []
test_angle_60_75_90 = []
test_angle_210_225 = []
test_angle_240_255_270 = []

for p in people_train:
    if p.guessAngle == 0 or p.guessAngle == 180:
        train_angle_0_180.append(p)
        
    elif p.guessAngle == 15 or p.guessAngle == 195:
        train_angle_15_195.append(p)
    
    elif p.guessAngle == 30 or p.guessAngle == 45:
        train_angle_30_45.append(p)
        
    elif p.guessAngle == 60 or p.guessAngle == 75 or p.guessAngle == 90:
        train_angle_60_75_90.append(p)
        
    elif p.guessAngle == 210 or p.guessAngle == 225:
        train_angle_210_225.append(p)
    else:
        train_angle_240_255_270.append(p)
        
for p in people_test:
    if p.guessAngle == 0 or p.guessAngle == 180:
        test_angle_0_180.append(p)
        
    elif p.guessAngle == 15 or p.guessAngle == 195:
        test_angle_15_195.append(p)
    
    elif p.guessAngle == 30 or p.guessAngle == 45:
        test_angle_30_45.append(p)
        
    elif p.guessAngle == 60 or p.guessAngle == 75 or p.guessAngle == 90:
        test_angle_60_75_90.append(p)
        
    elif p.guessAngle == 210 or p.guessAngle == 225:
        test_angle_210_225.append(p)
    else:
        test_angle_240_255_270.append(p)
        

# ------------------------------------------------------------------------------

# Training gnder with angle 0 and 180

In [80]:
block_gender_0_180 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_gender_0_180.add(Dense(2, activation = 'softmax'))
block_gender_0_180.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_gender_0_180.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [32]:
x_train = []
y_train = []

for p in train_angle_0_180:
    x_train.append(p.img)
    y_train.append(p.gender)
    
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

print(y_train.shape)

(9417, 2)


In [33]:
x_test = []
y_test = []

for p in test_angle_0_180:
    x_test.append(p.img)
    y_test.append(p.gender)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

print(y_test.shape)

(9416, 2)


In [ ]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(acc < 0.94):
        epochs += 5
        new_acc = train(block_gender_0_180)
        if new_acc > acc:
            acc = new_acc
            block_gender_0_180.save("model_MVLP/MVLP_gender_0_180.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
            print ("\n", b,"\n")

In [34]:
block_gender_0_180 = tf.keras.models.load_model("model_MVLP/MVLP_gender_0_180.h5")
test_loss, test_acc = block_gender_0_180.evaluate(x_test, y_test, verbose = 1)

295/295 [==============================] - 2s 7ms/step - loss: 0.7292 - accuracy: 0.9400


In [35]:
start = timeit.default_timer()

with tf.device("GPU:0"):
    for p in all_people:
        if p.guessAngle != 0 and p.guessAngle != 180:
            continue
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_gender = block_gender_0_180.predict(img)
        p.guessGender = np.argmax(predict_gender)

        b = "ID: "+ p.id + " Guess: " + str(p.guessGender)
        print (b, end="\r")

stop = timeit.default_timer()


# ------------------------------------------------------------------------------

# Training gnder with angle 15 and 195

In [76]:
block_gender_15_195 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_gender_15_195.add(Dense(2, activation = 'softmax'))
block_gender_15_195.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_gender_15_195.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [36]:
x_train = []
y_train = []

for p in train_angle_15_195:
    x_train.append(p.img)
    y_train.append(p.gender)
    
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

print(y_train.shape)

(9686, 2)


In [37]:
x_test = []
y_test = []

for p in test_angle_15_195:
    x_test.append(p.img)
    y_test.append(p.gender)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

print(y_test.shape)

(9698, 2)


In [ ]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(True):
        epochs += 5
        new_acc = train(block_gender_15_195)
        if new_acc > acc:
            acc = new_acc
            block_gender_15_195.save("model_MVLP/MVLP_gender_15_195.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
            print ("\n", b,"\n")

In [38]:
block_gender_15_195 = tf.keras.models.load_model("model_MVLP/MVLP_gender_15_195.h5")
test_loss, test_acc = block_gender_15_195.evaluate(x_test, y_test, verbose = 1)

304/304 [==============================] - 2s 7ms/step - loss: 0.8433 - accuracy: 0.9413


In [39]:
start = timeit.default_timer()

with tf.device("GPU:0"):
    for p in all_people:
        if p.guessAngle != 15 and p.guessAngle != 195:
            continue
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_gender = block_gender_15_195.predict(img)
        p.guessGender = np.argmax(predict_gender)

        b = "ID: "+ p.id + " Guess: " + str(p.guessGender)
        print (b, end="\r")

stop = timeit.default_timer()


# ------------------------------------------------------------------------------

# Training gnder with angle 30 and 45

In [31]:
block_gender_30_45 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_gender_30_45.add(Dense(2, activation = 'softmax'))
block_gender_30_45.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_gender_30_45.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [40]:
x_train = []
y_train = []

for p in train_angle_30_45:
    x_train.append(p.img)
    y_train.append(p.gender)
    
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

print(y_train.shape)

(10026, 2)


In [41]:
x_test = []
y_test = []

for p in test_angle_30_45:
    x_test.append(p.img)
    y_test.append(p.gender)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

print(y_test.shape)

(10051, 2)


In [ ]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(True):
        epochs += 5
        new_acc = train(block_gender_30_45)
        if new_acc > acc:
            acc = new_acc
            block_gender_30_45.save("model_MVLP/MVLP_gender_30_45.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
            print ("\n", b,"\n")

In [42]:
block_gender_30_45 = tf.keras.models.load_model("model_MVLP/MVLP_gender_30_45.h5")
test_loss, test_acc = block_gender_30_45.evaluate(x_test, y_test, verbose = 1)

315/315 [==============================] - 2s 7ms/step - loss: 0.5097 - accuracy: 0.9594


In [43]:
start = timeit.default_timer()

with tf.device("GPU:0"):
    for p in all_people:
        if p.guessAngle != 30 and p.guessAngle != 45:
            continue
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_gender = block_gender_30_45.predict(img)
        p.guessGender = np.argmax(predict_gender)

        b = "ID: "+ p.id + " Guess: " + str(p.guessGender)
        print (b, end="\r")

stop = timeit.default_timer()


# ------------------------------------------------------------------------------

# Training gnder with angle 60, 75 and 90

In [38]:
block_gender_60_75_90 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_gender_60_75_90.add(Dense(2, activation = 'softmax'))
block_gender_60_75_90.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_gender_60_75_90.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [44]:
x_train = []
y_train = []

for p in train_angle_60_75_90:
    x_train.append(p.img)
    y_train.append(p.gender)
    
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

print(y_train.shape)

(14816, 2)


In [45]:
x_test = []
y_test = []

for p in test_angle_60_75_90:
    x_test.append(p.img)
    y_test.append(p.gender)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

print(y_test.shape)

(14876, 2)


In [ ]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(True):
        epochs += 5
        new_acc = train(block_gender_60_75_90)
        if new_acc > acc:
            acc = new_acc
            block_gender_60_75_90.save("model_MVLP/MVLP_gender_60_75_90.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
            print ("\n", b,"\n")

In [46]:
block_gender_60_75_90 = tf.keras.models.load_model("model_MVLP/MVLP_gender_60_75_90.h5")
test_loss, test_acc = block_gender_60_75_90.evaluate(x_test, y_test, verbose = 1)

465/465 [==============================] - 3s 7ms/step - loss: 0.4984 - accuracy: 0.9584


In [47]:
start = timeit.default_timer()

with tf.device("GPU:0"):
    for p in all_people:
        if p.guessAngle != 60 and p.guessAngle != 75 and p.guessAngle != 90:
            continue
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_gender = block_gender_60_75_90.predict(img)
        p.guessGender = np.argmax(predict_gender)

        b = "ID: "+ p.id + " Guess: " + str(p.guessGender)
        print (b, end="\r")

stop = timeit.default_timer()


# ------------------------------------------------------------------------------

# Training gnder with angle 210 and 225

In [44]:
block_gender_210_225 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_gender_210_225.add(Dense(2, activation = 'softmax'))
block_gender_210_225.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_gender_210_225.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [48]:
x_train = []
y_train = []

for p in train_angle_210_225:
    x_train.append(p.img)
    y_train.append(p.gender)
    
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

print(y_train.shape)


x_test = []
y_test = []

for p in test_angle_210_225:
    x_test.append(p.img)
    y_test.append(p.gender)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

print(y_test.shape)

(10075, 2)
(10098, 2)


In [ ]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(True):
        epochs += 5
        new_acc = train(block_gender_210_225)
        if new_acc > acc:
            acc = new_acc
            block_gender_210_225.save("model_MVLP/MVLP_gender_210_225.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
            print ("\n", b,"\n")

In [49]:
block_gender_210_225 = tf.keras.models.load_model("model_MVLP/MVLP_gender_210_225.h5")
test_loss, test_acc = block_gender_210_225.evaluate(x_test, y_test, verbose = 1)

316/316 [==============================] - 2s 7ms/step - loss: 0.3143 - accuracy: 0.9568


In [50]:
start = timeit.default_timer()

with tf.device("GPU:0"):
    for p in all_people:
        if p.guessAngle != 210 and p.guessAngle != 225:
            continue
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_gender = block_gender_210_225.predict(img)
        p.guessGender = np.argmax(predict_gender)

        b = "ID: "+ p.id + " Guess: " + str(p.guessGender)
        print (b, end="\r")

stop = timeit.default_timer()


# ------------------------------------------------------------------------------

# Training gnder with angle 240, 255 and 270

In [52]:
block_gender_240_255_270 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_gender_240_255_270.add(Dense(2, activation = 'softmax'))
block_gender_240_255_270.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_gender_240_255_270.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [51]:
x_train = []
y_train = []

for p in train_angle_240_255_270:
    x_train.append(p.img)
    y_train.append(p.gender)
    
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

print(y_train.shape)


x_test = []
y_test = []

for p in test_angle_240_255_270:
    x_test.append(p.img)
    y_test.append(p.gender)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

print(y_test.shape)

(14779, 2)
(14876, 2)


In [ ]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(True):
        epochs += 5
        new_acc = train(block_gender_240_255_270)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_gender_240_255_270.save("model_MVLP/MVLP_gender_240_255_270.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

In [52]:
block_gender_240_255_270 = tf.keras.models.load_model("model_MVLP/MVLP_gender_240_255_270.h5")
test_loss, test_acc = block_gender_240_255_270.evaluate(x_test, y_test, verbose = 1)

465/465 [==============================] - 3s 7ms/step - loss: 0.7844 - accuracy: 0.9525: 3s - loss:


In [53]:
start = timeit.default_timer()

with tf.device("GPU:0"):
    for p in all_people:
        if p.guessAngle != 240 and p.guessAngle != 255 and p.guessAngle != 270:
            continue
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_gender = block_gender_240_255_270.predict(img)
        p.guessGender = np.argmax(predict_gender)

        b = "ID: "+ p.id + " Guess: " + str(p.guessGender)
        print (b, end="\r")

stop = timeit.default_timer()


# ------------------------------------------------------------------------------

# Predict gender by angles

In [8]:
train_female = []
train_male = []

test_female = []
test_male = []

for p in people_train:
    if p.guessGender == 0:
        train_female.append(p)
    else:
        train_male.append(p)
        
for p in people_test:
    if p.guessGender == 0:
        test_female.append(p)
    else:
        test_male.append(p)
    
    

# ------------------------------------------------------------------------------

# Training age with Female_0_180

In [9]:
def train(model):
    history = model.fit(x_train, y_train, epochs = 5)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 1)
    return test_acc

In [10]:
def to_cate(arr):
    min = 2
    max = 87
    y = []
    
    for age in arr:
        temp = [0]*(max-min+1)
        temp[age-2]+=1
        y.append(temp)
    
    return np.array(y, dtype='int32')
    
    

In [19]:
model = Sequential()
model.add(Conv2D(18, (7, 7), strides = 1, activation = 'relu', input_shape = (height, width, 1)))
model.add(MaxPooling2D(pool_size = (2,2), strides = 2))
model.add(BatchNormalization())

model.add(Conv2D(45, (5, 5), strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides = 2))
model.add(BatchNormalization())

model.add(Conv2D(60, (3, 3), strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), strides = 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.8))

model.add(Dense(86, activation = 'softmax'))
model.add(Dropout(0.5))
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
model.summary()

model.save("model_MVLP/MVLP_non_trained_layer3.h5")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 16, 60)       

In [14]:
x_train = []
y_train = []

for p in train_female:
    if p.age == '-':
        continue
    if p.guessAngle == 0 or p.guessAngle == 180:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_female:
    if p.age == '-':
        continue
    if p.guessAngle == 0 or p.guessAngle == 180:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4647, 86)
(4679, 86)


In [17]:
with tf.device("CPU:0"):
    block_age_F_0_180 = tf.keras.models.load_model('model_MVLP/MVLP_non_trained_layer3.h5')

In [ ]:
acc = 0.0885
epochs = 0

with tf.device("GPU:0"):
    while(epochs<1000):
        epochs += 5
        new_acc = train(block_age_F_0_180)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_F_0_180.save("model_MVLP/MVLP_age_F_0_180.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

In [16]:
block_age_F_0_180 = tf.keras.models.load_model("model_MVLP/MVLP_age_F_0_180.h5")
test_loss, test_acc = block_age_F_0_180.evaluate(x_test, y_test, verbose = 1)

147/147 [==============================] - 11s 7ms/step - loss: 3.8634 - accuracy: 0.0885 0s - los


# ------------------------------------------------------------------------------

# Training age with Male_0_180

In [11]:
x_train = []
y_train = []

for p in train_male:
    if p.age == '-':
        continue
    if p.guessAngle == 0 or p.guessAngle == 180:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_male:
    if p.age == '-':
        continue
    if p.guessAngle == 0 or p.guessAngle == 180:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4584, 86)
(4532, 86)


In [23]:
with tf.device("CPU:0"):
    block_age_M_0_180 = tf.keras.models.load_model('model_MVLP/MVLP_non_trained_layer3.h5')

In [24]:
acc = 0.0916
epochs = 0

with tf.device("GPU:0"):
    while(epochs<800):
        epochs += 5
        new_acc = train(block_age_M_0_180)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_M_0_180.save("model_MVLP/MVLP_age_M_0_180.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
144/144 [==============================] - 6s 36ms/step - loss: 10.6277 - accuracy: 0.0325
Epoch 2/5
144/144 [==============================] - 5s 35ms/step - loss: 10.2258 - accuracy: 0.0537
Epoch 3/5
144/144 [==============================] - 5s 35ms/step - loss: 9.9828 - accuracy: 0.0650
Epoch 4/5
144/144 [==============================] - 5s 35ms/step - loss: 9.7602 - accuracy: 0.0702
Epoch 5/5
142/142 [==============================] - 2s 11ms/step - loss: 5.3365 - accuracy: 0.0644

 Epochs: 5 

Epoch 1/5
144/144 [==============================] - 5s 35ms/step - loss: 9.5672 - accuracy: 0.0801
Epoch 2/5
144/144 [==============================] - 5s 35ms/step - loss: 9.6160 - accuracy: 0.0938
Epoch 3/5
144/144 [==============================] - 5s 36ms/step - loss: 9.4077 - accuracy: 0.0877
Epoch 4/5
144/144 [==============================] - 5s 35ms/step - loss: 9.3631 - accuracy: 0.0973
Epoch 5/5
142/142 [==============================] - 2s 11ms/step - loss: 3.9533 - a

142/142 [==============================] - 2s 11ms/step - loss: 7.4134 - accuracy: 0.0841

 Epochs: 135 

Epoch 1/5
144/144 [==============================] - 5s 36ms/step - loss: 8.1296 - accuracy: 0.4640
Epoch 2/5
144/144 [==============================] - 5s 38ms/step - loss: 8.0077 - accuracy: 0.4780
Epoch 3/5
144/144 [==============================] - 5s 38ms/step - loss: 8.2071 - accuracy: 0.4638
Epoch 4/5
144/144 [==============================] - 5s 37ms/step - loss: 8.3671 - accuracy: 0.4557
Epoch 5/5
142/142 [==============================] - 2s 12ms/step - loss: 7.4343 - accuracy: 0.0845

 Epochs: 140 

Epoch 1/5
144/144 [==============================] - 5s 37ms/step - loss: 8.5051 - accuracy: 0.4422
Epoch 2/5
144/144 [==============================] - 5s 37ms/step - loss: 8.0294 - accuracy: 0.4719
Epoch 3/5
144/144 [==============================] - 5s 37ms/step - loss: 8.2684 - accuracy: 0.4559
Epoch 4/5
144/144 [==============================] - 5s 36ms/step - loss: 8.21

144/144 [==============================] - 6s 38ms/step - loss: 8.0385 - accuracy: 0.4849
Epoch 4/5
144/144 [==============================] - 5s 37ms/step - loss: 8.0360 - accuracy: 0.4834
Epoch 5/5
142/142 [==============================] - 2s 12ms/step - loss: 8.3564 - accuracy: 0.0816

 Epochs: 205 

Epoch 1/5
144/144 [==============================] - 5s 38ms/step - loss: 7.9912 - accuracy: 0.4865
Epoch 2/5
144/144 [==============================] - 5s 37ms/step - loss: 8.0731 - accuracy: 0.4825
Epoch 3/5
144/144 [==============================] - 5s 38ms/step - loss: 8.0847 - accuracy: 0.4784
Epoch 4/5
144/144 [==============================] - 5s 38ms/step - loss: 8.1719 - accuracy: 0.4721
Epoch 5/5
142/142 [==============================] - 2s 12ms/step - loss: 7.9265 - accuracy: 0.0757

 Epochs: 210 

Epoch 1/5
144/144 [==============================] - 5s 38ms/step - loss: 8.3325 - accuracy: 0.4594
Epoch 2/5
144/144 [==============================] - 5s 38ms/step - loss: 8.25

144/144 [==============================] - 6s 38ms/step - loss: 8.3428 - accuracy: 0.4644
Epoch 4/5
144/144 [==============================] - 6s 38ms/step - loss: 8.2142 - accuracy: 0.4708
Epoch 5/5
142/142 [==============================] - 2s 12ms/step - loss: 8.9718 - accuracy: 0.0810

 Epochs: 340 

Epoch 1/5
144/144 [==============================] - 6s 38ms/step - loss: 8.0787 - accuracy: 0.4832
Epoch 2/5
144/144 [==============================] - 6s 38ms/step - loss: 8.1479 - accuracy: 0.4782
Epoch 3/5
144/144 [==============================] - 6s 39ms/step - loss: 8.1675 - accuracy: 0.4791
Epoch 4/5
144/144 [==============================] - 6s 38ms/step - loss: 8.1729 - accuracy: 0.4771
Epoch 5/5
142/142 [==============================] - 2s 13ms/step - loss: 9.3435 - accuracy: 0.0779

 Epochs: 345 

Epoch 1/5
144/144 [==============================] - 5s 38ms/step - loss: 8.2205 - accuracy: 0.4758
Epoch 2/5
144/144 [==============================] - 6s 38ms/step - loss: 8.09

142/142 [==============================] - 2s 12ms/step - loss: 9.4334 - accuracy: 0.0763

 Epochs: 405 

Epoch 1/5
144/144 [==============================] - 6s 39ms/step - loss: 8.1534 - accuracy: 0.4815
Epoch 2/5
144/144 [==============================] - 6s 40ms/step - loss: 8.0568 - accuracy: 0.4911
Epoch 3/5
144/144 [==============================] - 5s 38ms/step - loss: 7.9454 - accuracy: 0.4961
Epoch 4/5
144/144 [==============================] - 6s 39ms/step - loss: 8.1692 - accuracy: 0.4839
Epoch 5/5
142/142 [==============================] - 2s 12ms/step - loss: 9.7824 - accuracy: 0.0761

 Epochs: 410 

Epoch 1/5
144/144 [==============================] - 5s 38ms/step - loss: 7.9733 - accuracy: 0.4950
Epoch 2/5
144/144 [==============================] - 6s 39ms/step - loss: 8.0524 - accuracy: 0.4935
Epoch 3/5
144/144 [==============================] - 6s 39ms/step - loss: 7.9617 - accuracy: 0.4959
Epoch 4/5
144/144 [==============================] - 5s 38ms/step - loss: 8.09

...

144/144 [==============================] - 6s 41ms/step - loss: 7.9528 - accuracy: 0.4987
Epoch 3/5
144/144 [==============================] - 6s 42ms/step - loss: 8.3646 - accuracy: 0.4734
Epoch 4/5
144/144 [==============================] - 6s 41ms/step - loss: 8.2341 - accuracy: 0.4834
Epoch 5/5
142/142 [==============================] - 2s 16ms/step - loss: 9.8877 - accuracy: 0.0812A: 2s - 

 Epochs: 610 

Epoch 1/5
144/144 [==============================] - 6s 42ms/step - loss: 8.2210 - accuracy: 0.4817
Epoch 2/5
144/144 [==============================] - 6s 41ms/step - loss: 7.9075 - accuracy: 0.5031
Epoch 3/5
144/144 [==============================] - 6s 42ms/step - loss: 7.9864 - accuracy: 0.4980
Epoch 4/5
144/144 [==============================] - 6s 41ms/step - loss: 8.1658 - accuracy: 0.4858
Epoch 5/5
142/142 [==============================] - 2s 14ms/step - loss: 9.7859 - accuracy: 0.0744 0s - los

 Epochs: 615 

Epoch 1/5
144/144 [==============================] - 6s 41ms/

142/142 [==============================] - 2s 15ms/step - loss: 10.2639 - accuracy: 0.0702: 0s - loss: 10.2341 - accurac

 Epochs: 740 

Epoch 1/5
144/144 [==============================] - 6s 44ms/step - loss: 8.0136 - accuracy: 0.4974
Epoch 2/5
144/144 [==============================] - 7s 46ms/step - loss: 7.8706 - accuracy: 0.5065
Epoch 3/5
144/144 [==============================] - 6s 42ms/step - loss: 8.0558 - accuracy: 0.4928
Epoch 4/5
144/144 [==============================] - 7s 46ms/step - loss: 8.1441 - accuracy: 0.4871
Epoch 5/5
142/142 [==============================] - 2s 15ms/step - loss: 10.5225 - accuracy: 0.0682

 Epochs: 745 

Epoch 1/5
144/144 [==============================] - 6s 45ms/step - loss: 8.1826 - accuracy: 0.4863
Epoch 2/5
144/144 [==============================] - 6s 45ms/step - loss: 8.0534 - accuracy: 0.4928
Epoch 3/5
144/144 [==============================] - 6s 43ms/step - loss: 8.1239 - accuracy: 0.4873
Epoch 4/5
144/144 [===========================

144/144 [==============================] - 6s 42ms/step - loss: 8.0227 - accuracy: 0.4961
Epoch 3/5
144/144 [==============================] - 6s 44ms/step - loss: 8.2161 - accuracy: 0.4839
Epoch 4/5
144/144 [==============================] - 6s 44ms/step - loss: 8.1205 - accuracy: 0.4889
Epoch 5/5
142/142 [==============================] - 2s 15ms/step - loss: 10.3296 - accuracy: 0.0836

 Epochs: 810 

Epoch 1/5
144/144 [==============================] - 6s 44ms/step - loss: 7.9120 - accuracy: 0.5013
Epoch 2/5
144/144 [==============================] - 7s 46ms/step - loss: 8.1259 - accuracy: 0.4884
Epoch 3/5
144/144 [==============================] - 7s 48ms/step - loss: 7.9813 - accuracy: 0.5007
Epoch 4/5
144/144 [==============================] - 8s 53ms/step - loss: 8.0551 - accuracy: 0.4937
Epoch 5/5
142/142 [==============================] - 2s 16ms/step - loss: 10.9451 - accuracy: 0.0821

 Epochs: 815 

Epoch 1/5
144/144 [==============================] - 7s 49ms/step - loss: 7.

144/144 [==============================] - 8s 59ms/step - loss: 8.3116 - accuracy: 0.4812
Epoch 3/5
144/144 [==============================] - 9s 60ms/step - loss: 8.0968 - accuracy: 0.4924
Epoch 4/5
144/144 [==============================] - 9s 62ms/step - loss: 8.2300 - accuracy: 0.4843
Epoch 5/5
142/142 [==============================] - 3s 21ms/step - loss: 10.6033 - accuracy: 0.0847: 2

 Epochs: 940 

Epoch 1/5
144/144 [==============================] - 10s 66ms/step - loss: 8.1551 - accuracy: 0.4897
Epoch 2/5
144/144 [==============================] - 8s 55ms/step - loss: 8.2027 - accuracy: 0.4873
Epoch 3/5
144/144 [==============================] - 8s 56ms/step - loss: 8.0584 - accuracy: 0.4950
Epoch 4/5
144/144 [==============================] - 9s 62ms/step - loss: 8.1755 - accuracy: 0.4871
Epoch 5/5
142/142 [==============================] - 3s 21ms/step - loss: 10.5569 - accuracy: 0.0788

 Epochs: 945 

Epoch 1/5
144/144 [==============================] - 9s 60ms/step - loss

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[32,128,88,1] and type uint8 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[32,128,88,1] and type uint8 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[sequential_2/conv2d_7/BiasAdd/ReadVariableOp/_40]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 d

In [12]:
block_age_M_0_180 = tf.keras.models.load_model("model_MVLP/MVLP_age_M_0_180.h5")
test_loss, test_acc = block_age_M_0_180.evaluate(x_test, y_test, verbose = 1)

142/142 [==============================] - 5s 8ms/step - loss: 4.5586 - accuracy: 0.0916


# ------------------------------------------------------------------------------

# Training age with Female_15_195

In [38]:
x_train = []
y_train = []

for p in train_female:
    if p.age == '-':
        continue
    if p.guessAngle == 15 or p.guessAngle == 195:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_female:
    if p.age == '-':
        continue
    if p.guessAngle == 15 or p.guessAngle == 195:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4767, 86)
(4831, 86)


In [39]:
block_age_F_15_195 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_age_F_15_195.add(Dense(86, activation = 'softmax'))
block_age_F_15_195.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_age_F_15_195.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [40]:
acc = 0.0811
epochs = 0

with tf.device("GPU:0"):
    while(epochs<100):
        epochs += 5
        new_acc = train(block_age_F_15_195)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_F_15_195.save("model_MVLP/MVLP_age_F_15_195.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
149/149 [==============================] - 4s 21ms/step - loss: 14.5699 - accuracy: 0.0335:
Epoch 2/5
149/149 [==============================] - 3s 20ms/step - loss: 4.0030 - accuracy: 0.0548 0s - loss: 4.0089 - accu - ETA: 0s - loss: 4.0049 - accuracy: 
Epoch 3/5
149/149 [==============================] - 3s 20ms/step - loss: 3.7609 - accuracy: 0.0829
Epoch 4/5
149/149 [==============================] - 3s 20ms/step - loss: 3.6547 - accuracy: 0.0855
Epoch 5/5
151/151 [==============================] - 1s 7ms/step - loss: 3.7550 - accuracy: 0.0758

 Epochs: 5 Acc: 0.07576071470975876 

Epoch 1/5
149/149 [==============================] - 3s 21ms/step - loss: 3.5306 - accuracy: 0.0944
Epoch 2/5
149/149 [==============================] - 3s 21ms/step - loss: 3.5114 - accuracy: 0.0969
Epoch 3/5
149/149 [==============================] - 3s 21ms/step - loss: 3.4306 - accuracy: 0.1137
Epoch 4/5
149/149 [==============================] - 3s 21ms/step - loss: 3.3952 - accuracy: 0.11

151/151 [==============================] - 1s 7ms/step - loss: 19.5633 - accuracy: 0.0646A: 0s - loss: 20.1243 

 Epochs: 85 

Epoch 1/5
149/149 [==============================] - 3s 22ms/step - loss: 0.0354 - accuracy: 0.9912 0s - loss: 0.0356 - accuracy
Epoch 2/5
149/149 [==============================] - 3s 22ms/step - loss: 0.0396 - accuracy: 0.9906 2s - loss: 0.0405 - accu - ETA
Epoch 3/5
149/149 [==============================] - 4s 24ms/step - loss: 0.0657 - accuracy: 0.9807
Epoch 4/5
149/149 [==============================] - 3s 21ms/step - loss: 0.1847 - accuracy: 0.9492 0s - loss: 0.1834 - accura
Epoch 5/5
151/151 [==============================] - 1s 8ms/step - loss: 19.6629 - accuracy: 0.0596

 Epochs: 90 

Epoch 1/5
149/149 [==============================] - 3s 21ms/step - loss: 0.2948 - accuracy: 0.9188 0s - loss:
Epoch 2/5
149/149 [==============================] - 3s 21ms/step - loss: 0.1253 - accuracy: 0.9635 1s - loss: 0.1265 - accuracy: 0. - ETA: 0s - loss: 0.1246 - 

In [41]:
block_age_F_15_195 = tf.keras.models.load_model("model_MVLP/MVLP_age_F_15_195.h5")
test_loss, test_acc = block_age_F_15_195.evaluate(x_test, y_test, verbose = 1)

151/151 [==============================] - 1s 7ms/step - loss: 3.7875 - accuracy: 0.0811


# ------------------------------------------------------------------------------

# Training age with Male_15_195

In [42]:
x_train = []
y_train = []

for p in train_male:
    if p.age == '-':
        continue
    if p.guessAngle == 15 or p.guessAngle == 195:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_male:
    if p.age == '-':
        continue
    if p.guessAngle == 15 or p.guessAngle == 195:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4727, 86)
(4663, 86)


In [43]:
block_age_M_15_195 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

block_age_M_15_195.add(Dense(86, activation = 'softmax'))
block_age_M_15_195.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_age_M_15_195.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [ ]:
acc = 0.0843
epochs = 0

with tf.device("GPU:0"):
    while(epochs<100):
        epochs += 5
        new_acc = train(block_age_M_15_195)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_M_15_195.save("model_MVLP/MVLP_age_M_15_195.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

In [45]:
block_age_M_15_195 = tf.keras.models.load_model("model_MVLP/MVLP_age_M_15_195.h5")
test_loss, test_acc = block_age_M_15_195.evaluate(x_test, y_test, verbose = 1)

146/146 [==============================] - 1s 8ms/step - loss: 3.8192 - accuracy: 0.0843


# ------------------------------------------------------------------------------

# Training age with Female_30_45

In [46]:
x_train = []
y_train = []

for p in train_female:
    if p.age == '-':
        continue
    if p.guessAngle == 30 or p.guessAngle == 45:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_female:
    if p.age == '-':
        continue
    if p.guessAngle == 30 or p.guessAngle == 45:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4914, 86)
(4999, 86)


In [50]:
with tf.device("CPU:0"):
    block_age_F_30_45 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_F_30_45.add(Dense(86, activation = 'softmax'))
    block_age_F_30_45.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_F_30_45.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [ ]:
acc = 0.0790
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_F_30_45)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_F_30_45.save("model_MVLP/MVLP_age_F_30_45.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

In [52]:
block_age_F_30_45 = tf.keras.models.load_model("model_MVLP/MVLP_age_F_30_45.h5")
test_loss, test_acc = block_age_F_30_45.evaluate(x_test, y_test, verbose = 1)

157/157 [==============================] - 1s 8ms/step - loss: 3.8313 - accuracy: 0.0848


# ------------------------------------------------------------------------------

# Training age with Male_30_45

In [53]:
x_train = []
y_train = []

for p in train_male:
    if p.age == '-':
        continue
    if p.guessAngle == 30 or p.guessAngle == 45:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_male:
    if p.age == '-':
        continue
    if p.guessAngle == 30 or p.guessAngle == 45:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4912, 86)
(4826, 86)


In [54]:
with tf.device("CPU:0"):
    block_age_M_30_45 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_M_30_45.add(Dense(86, activation = 'softmax'))
    block_age_M_30_45.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_M_30_45.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [ ]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_M_30_45)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_M_30_45.save("model_MVLP/MVLP_age_M_30_45.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

In [56]:
block_age_M_30_45 = tf.keras.models.load_model("model_MVLP/MVLP_age_M_30_45.h5")
test_loss, test_acc = block_age_M_30_45.evaluate(x_test, y_test, verbose = 1)

151/151 [==============================] - 1s 8ms/step - loss: 3.8162 - accuracy: 0.0901


# ------------------------------------------------------------------------------

# Training age with Female_60_75_90

In [68]:
x_train = []
y_train = []

for p in train_female:
    if p.age == '-':
        continue
    if p.guessAngle == 60 or p.guessAngle == 75 or p.guessAngle == 90:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_female:
    if p.age == '-':
        continue
    if p.guessAngle == 60 or p.guessAngle == 75 or p.guessAngle == 90:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(7240, 86)
(7335, 86)


In [69]:
with tf.device("CPU:0"):
    block_age_F_60_75_90 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_F_60_75_90.add(Dense(86, activation = 'softmax'))
    block_age_F_60_75_90.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_F_60_75_90.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [70]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_F_60_75_90)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_F_60_75_90.save("model_MVLP/MVLP_age_F_60_75_90.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
227/227 [==============================] - 25s 108ms/step - loss: 11.5441 - accuracy: 0.0425
Epoch 2/5
227/227 [==============================] - 25s 109ms/step - loss: 3.6531 - accuracy: 0.0936
Epoch 3/5
227/227 [==============================] - 24s 106ms/step - loss: 3.4718 - accuracy: 0.1054
Epoch 4/5
227/227 [==============================] - 24s 104ms/step - loss: 3.3832 - accuracy: 0.1154
Epoch 5/5
230/230 [==============================] - 4s 17ms/step - loss: 3.6514 - accuracy: 0.0852

 Epochs: 5 Acc: 0.08520790934562683 

Epoch 1/5
227/227 [==============================] - 26s 113ms/step - loss: 3.1333 - accuracy: 0.1579
Epoch 2/5
227/227 [==============================] - 25s 109ms/step - loss: 3.0134 - accuracy: 0.1849
Epoch 3/5
227/227 [==============================] - 24s 107ms/step - loss: 2.9162 - accuracy: 0.2048
Epoch 4/5
227/227 [==============================] - 25s 108ms/step - loss: 2.7906 - accuracy: 0.2297
Epoch 5/5
230/230 [=========================

In [71]:
block_age_F_60_75_90 = tf.keras.models.load_model("model_MVLP/MVLP_age_F_60_75_90.h5")
test_loss, test_acc = block_age_F_60_75_90.evaluate(x_test, y_test, verbose = 1)

230/230 [==============================] - 2s 8ms/step - loss: 3.6514 - accuracy: 0.0852


# ------------------------------------------------------------------------------

# Training age with Male_60_75_90

In [72]:
x_train = []
y_train = []

for p in train_male:
    if p.age == '-':
        continue
    if p.guessAngle == 60 or p.guessAngle == 75 or p.guessAngle == 90:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_male:
    if p.age == '-':
        continue
    if p.guessAngle == 60 or p.guessAngle == 75 or p.guessAngle == 90:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(7282, 86)
(7220, 86)


In [73]:
with tf.device("CPU:0"):
    block_age_M_60_75_90 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_M_60_75_90.add(Dense(86, activation = 'softmax'))
    block_age_M_60_75_90.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_M_60_75_90.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [74]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_M_60_75_90)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_M_60_75_90.save("model_MVLP/MVLP_age_M_60_75_90.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
228/228 [==============================] - 28s 119ms/step - loss: 11.7526 - accuracy: 0.0371
Epoch 2/5
228/228 [==============================] - 28s 124ms/step - loss: 3.6136 - accuracy: 0.0802
Epoch 3/5
228/228 [==============================] - 28s 123ms/step - loss: 3.4540 - accuracy: 0.1107
Epoch 4/5
228/228 [==============================] - 26s 115ms/step - loss: 3.3837 - accuracy: 0.1071
Epoch 5/5
226/226 [==============================] - 4s 18ms/step - loss: 3.8547 - accuracy: 0.0688

 Epochs: 5 Acc: 0.06883656233549118 

Epoch 1/5
228/228 [==============================] - 29s 125ms/step - loss: 3.1737 - accuracy: 0.1399
Epoch 2/5
228/228 [==============================] - 28s 125ms/step - loss: 3.0752 - accuracy: 0.1560
Epoch 3/5
228/228 [==============================] - 27s 117ms/step - loss: 2.9715 - accuracy: 0.1752
Epoch 4/5
228/228 [==============================] - 25s 108ms/step - loss: 2.8665 - accuracy: 0.2031
Epoch 5/5
226/226 [=========================

In [75]:
block_age_M_60_75_90 = tf.keras.models.load_model("model_MVLP/MVLP_age_M_60_75_90.h5")
test_loss, test_acc = block_age_M_60_75_90.evaluate(x_test, y_test, verbose = 1)

226/226 [==============================] - 2s 8ms/step - loss: 4.8616 - accuracy: 0.0868


# ------------------------------------------------------------------------------

# Training age with Female_210_225

In [92]:
x_train = []
y_train = []

for p in train_female:
    if p.age == '-':
        continue
    if p.guessAngle == 210 or p.guessAngle == 225:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_female:
    if p.age == '-':
        continue
    if p.guessAngle == 210 or p.guessAngle == 225:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4936, 86)
(5017, 86)


In [93]:
with tf.device("CPU:0"):
    block_age_F_210_225 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_F_210_225.add(Dense(86, activation = 'softmax'))
    block_age_F_210_225.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_F_210_225.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [94]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_F_210_225)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_F_210_225.save("model_MVLP/MVLP_age_F_210_225.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
155/155 [==============================] - 17s 109ms/step - loss: 14.7830 - accuracy: 0.0391
Epoch 2/5
155/155 [==============================] - 20s 128ms/step - loss: 3.8437 - accuracy: 0.0644
Epoch 3/5
155/155 [==============================] - 20s 127ms/step - loss: 3.6227 - accuracy: 0.0851
Epoch 4/5
155/155 [==============================] - 20s 131ms/step - loss: 3.5313 - accuracy: 0.0915
Epoch 5/5
157/157 [==============================] - 3s 19ms/step - loss: 3.8358 - accuracy: 0.0775

 Epochs: 5 Acc: 0.07753637433052063 

Epoch 1/5
155/155 [==============================] - 18s 119ms/step - loss: 3.4090 - accuracy: 0.1114
Epoch 2/5
155/155 [==============================] - 17s 113ms/step - loss: 3.3535 - accuracy: 0.1124
Epoch 3/5
155/155 [==============================] - 18s 118ms/step - loss: 3.2993 - accuracy: 0.1187
Epoch 4/5
155/155 [==============================] - 17s 111ms/step - loss: 3.2367 - accuracy: 0.1309
Epoch 5/5
157/157 [=========================

In [95]:
block_age_F_210_225 = tf.keras.models.load_model("model_MVLP/MVLP_age_F_210_225.h5")
test_loss, test_acc = block_age_F_210_225.evaluate(x_test, y_test, verbose = 1)

157/157 [==============================] - 1s 7ms/step - loss: 3.9697 - accuracy: 0.0859


# ------------------------------------------------------------------------------

# Training age with Male_210_225

In [80]:
x_train = []
y_train = []

for p in train_male:
    if p.age == '-':
        continue
    if p.guessAngle == 210 or p.guessAngle == 225:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_male:
    if p.age == '-':
        continue
    if p.guessAngle == 210 or p.guessAngle == 225:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(4937, 86)
(4865, 86)


In [81]:
with tf.device("CPU:0"):
    block_age_M_210_225 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_M_210_225.add(Dense(86, activation = 'softmax'))
    block_age_M_210_225.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_M_210_225.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [82]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_M_210_225)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_M_210_225.save("model_MVLP/MVLP_age_M_210_225.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
155/155 [==============================] - 16s 102ms/step - loss: 16.9128 - accuracy: 0.0410
Epoch 2/5
155/155 [==============================] - 16s 102ms/step - loss: 3.6953 - accuracy: 0.0688
Epoch 3/5
155/155 [==============================] - 16s 103ms/step - loss: 3.5656 - accuracy: 0.0894
Epoch 4/5
155/155 [==============================] - 16s 103ms/step - loss: 3.4355 - accuracy: 0.1193
Epoch 5/5
153/153 [==============================] - 3s 17ms/step - loss: 3.6945 - accuracy: 0.0828

 Epochs: 5 Acc: 0.0828365907073021 

Epoch 1/5
155/155 [==============================] - 16s 104ms/step - loss: 3.2739 - accuracy: 0.1298
Epoch 2/5
155/155 [==============================] - 16s 103ms/step - loss: 3.2176 - accuracy: 0.1373
Epoch 3/5
155/155 [==============================] - 16s 102ms/step - loss: 3.1484 - accuracy: 0.1525
Epoch 4/5
155/155 [==============================] - 16s 104ms/step - loss: 3.0670 - accuracy: 0.1665
Epoch 5/5
153/153 [==========================

In [83]:
block_age_M_210_225 = tf.keras.models.load_model("model_MVLP/MVLP_age_M_210_225.h5")
test_loss, test_acc = block_age_M_210_225.evaluate(x_test, y_test, verbose = 1)

153/153 [==============================] - 1s 7ms/step - loss: 15.2663 - accuracy: 0.0876


# ------------------------------------------------------------------------------

# Training age with Female_240_255_270

In [84]:
x_train = []
y_train = []

for p in train_female:
    if p.age == '-':
        continue
    if p.guessAngle == 240 or p.guessAngle == 255 or p.guessAngle == 270:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_female:
    if p.age == '-':
        continue
    if p.guessAngle == 240 or p.guessAngle == 255 or p.guessAngle == 270:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(7230, 86)
(7279, 86)


In [85]:
with tf.device("CPU:0"):
    block_age_F_240_255_270 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_F_240_255_270.add(Dense(86, activation = 'softmax'))
    block_age_F_240_255_270.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_F_240_255_270.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [86]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_F_240_255_270)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_F_240_255_270.save("model_MVLP/MVLP_age_F_240_255_270.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
226/226 [==============================] - 23s 100ms/step - loss: 11.5074 - accuracy: 0.0442
Epoch 2/5
226/226 [==============================] - 22s 99ms/step - loss: 3.7102 - accuracy: 0.0813
Epoch 3/5
226/226 [==============================] - 23s 101ms/step - loss: 3.4509 - accuracy: 0.1080
Epoch 4/5
226/226 [==============================] - 23s 100ms/step - loss: 3.3081 - accuracy: 0.1342
Epoch 5/5
228/228 [==============================] - 4s 17ms/step - loss: 3.6908 - accuracy: 0.0812

 Epochs: 5 Acc: 0.08119247108697891 

Epoch 1/5
226/226 [==============================] - 22s 99ms/step - loss: 3.0968 - accuracy: 0.1686
Epoch 2/5
226/226 [==============================] - 23s 100ms/step - loss: 3.0006 - accuracy: 0.1846
Epoch 3/5
226/226 [==============================] - 22s 99ms/step - loss: 2.8635 - accuracy: 0.2181
Epoch 4/5
226/226 [==============================] - 22s 100ms/step - loss: 2.7505 - accuracy: 0.2358
Epoch 5/5
228/228 [============================

In [87]:
block_age_F_240_255_270 = tf.keras.models.load_model("model_MVLP/MVLP_age_F_240_255_270.h5")
test_loss, test_acc = block_age_F_240_255_270.evaluate(x_test, y_test, verbose = 1)

228/228 [==============================] - 2s 7ms/step - loss: 4.2309 - accuracy: 0.0819


# ------------------------------------------------------------------------------

# Training age with Male_240_255_270

In [88]:
x_train = []
y_train = []

for p in train_male:
    if p.age == '-':
        continue
    if p.guessAngle == 240 or p.guessAngle == 255 or p.guessAngle == 270:
        x_train.append(p.img)
        y_train.append(p.age)
        
x_train = np.array(x_train).reshape(-1, height, width, 1)
y_train = to_cate(y_train)
print(y_train.shape)

x_test = []
y_test = []

for p in test_male:
    if p.age == '-':
        continue
    if p.guessAngle == 240 or p.guessAngle == 255 or p.guessAngle == 270:
        x_test.append(p.img)
        y_test.append(p.age)
    
x_test = np.array(x_test).reshape(-1, height, width, 1)
y_test = to_cate(y_test)
print(y_test.shape)

(7254, 86)
(7275, 86)


In [89]:
with tf.device("CPU:0"):
    block_age_M_240_255_270 = tf.keras.models.load_model('model_MVLP/MVLP_without_output_layer.h5')

    block_age_M_240_255_270.add(Dense(86, activation = 'softmax'))
    block_age_M_240_255_270.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    block_age_M_240_255_270.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 122, 82, 18)       900       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 41, 18)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 61, 41, 18)        72        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 57, 37, 45)        20295     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 18, 45)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 18, 45)        180       
_________________________________________________________________
flatten_2 (Flatten)          (None, 22680)            

In [90]:
acc = 0
epochs = 0

with tf.device("GPU:0"):
    while(epochs<50):
        epochs += 5
        new_acc = train(block_age_M_240_255_270)
        b = "Epochs: "+ str(epochs)
        if new_acc > acc:
            acc = new_acc
            block_age_M_240_255_270.save("model_MVLP/MVLP_age_M_240_255_270.h5")
            b = "Epochs: "+ str(epochs) + " Acc: " + str(acc)
        print ("\n", b,"\n")

Epoch 1/5
227/227 [==============================] - 24s 102ms/step - loss: 12.0036 - accuracy: 0.0421
Epoch 2/5
227/227 [==============================] - 23s 101ms/step - loss: 3.6172 - accuracy: 0.0711
Epoch 3/5
227/227 [==============================] - 23s 101ms/step - loss: 3.4892 - accuracy: 0.0831
Epoch 4/5
227/227 [==============================] - 23s 103ms/step - loss: 3.3682 - accuracy: 0.0987
Epoch 5/5
228/228 [==============================] - 4s 17ms/step - loss: 3.6099 - accuracy: 0.0841

 Epochs: 5 Acc: 0.08412370830774307 

Epoch 1/5
227/227 [==============================] - 24s 104ms/step - loss: 3.2360 - accuracy: 0.1245
Epoch 2/5
227/227 [==============================] - 25s 108ms/step - loss: 3.1930 - accuracy: 0.1322
Epoch 3/5
227/227 [==============================] - 24s 106ms/step - loss: 3.1499 - accuracy: 0.1431
Epoch 4/5
227/227 [==============================] - 24s 106ms/step - loss: 3.0715 - accuracy: 0.1596
Epoch 5/5
228/228 [=========================

In [91]:
block_age_M_240_255_270 = tf.keras.models.load_model("model_MVLP/MVLP_age_M_240_255_270.h5")
test_loss, test_acc = block_age_M_240_255_270.evaluate(x_test, y_test, verbose = 1)

228/228 [==============================] - 2s 8ms/step - loss: 3.7813 - accuracy: 0.0874
